In [10]:
import numpy as np
import os

In [11]:
Y=np.load('samples.npy')
X=np.roll(Y,1,axis=1)
X[:,0,:]=0
np.array_equal(Y[:,0:94,:],X[:,1:95,:]) 

True

In [12]:
from tensorflow.keras.layers import Input,LSTM,Dropout,Dense
from tensorflow.keras.models import Model

In [13]:
#input layer
encoder_input=Input(shape=(None,X.shape[2]))
#LSTM Layer 1
encoder_LSTM1=LSTM(1024,return_sequences=True)
encoder_outputs=encoder_LSTM1(encoder_input)
#dropout layer
encoder_outputs=Dropout(0.3)(encoder_outputs)
#LSTM layer 2
encoder_LSTM2=LSTM(1024,return_state=True)
encoder_outputs,encoder_h,encoder_c=encoder_LSTM2(encoder_outputs)
#encoder states
encoder_states=[encoder_h,encoder_c]

In [14]:
###Building the decoder
#input layer
decoder_input=Input(shape=(None,X.shape[2]))
#LSTM layer 1
decoder_LSTM1=LSTM(1024,return_sequences=True)
decoder_out=decoder_LSTM1(decoder_input,initial_state=encoder_states)
#LSTM layer 2
decoder_LSTM2=LSTM(1024,return_sequences=True,return_state=True)
decoder_out,_,_=decoder_LSTM2(decoder_out)
#softmax activation layer
decoder_dense=Dense(96,activation='softmax')
decoder_out=decoder_dense(decoder_out)

In [19]:
#compile
model=Model(inputs=[encoder_input,decoder_input],outputs=[decoder_out])
print(model.summary())
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
model.save('autoencodermodel.h5')


Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, 96)]   0                                            
__________________________________________________________________________________________________
lstm_4 (LSTM)                   (None, None, 1024)   4591616     input_3[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, None, 1024)   0           lstm_4[0][0]                     
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None, 96)]   0                                            
____________________________________________________________________________________________

In [9]:
#fit the model
from tensorflow.keras.callbacks import ModelCheckpoint
filepath="bestweight.hdf5"
checkpoint=ModelCheckpoint(filepath,monitor='loss',verbose=0,save_best_only=True)
history=model.fit(x=[X,X],y=Y,batch_size=64,epochs=50,validation_split=0.2,callbacks=[checkpoint])

Train on 99603 samples, validate on 24901 samples
Epoch 1/50
   64/99603 [..............................] - ETA: 3:15:29WARNING:tensorflow:Can save best model only with loss available, skipping.


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy vs validation accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()